# Top Bottom

In [35]:
from devito import Eq, Grid, TimeFunction, Operator

grid = Grid(shape=(3, 3))
u = TimeFunction(name='u', grid=grid)
u.data

Data([[[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]],

      [[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]], dtype=float32)

In [36]:
from devito import configuration
configuration['openmp'] = 0  # We don't care about OpenMP in this example

eq = Eq(u.forward, u+1)
op = Operator(eq, dle='noop')
print(op)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict u_vec, const int time_M, const int time_m, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m)
{
  float (*restrict u)[u_vec->size[1]][u_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]]) u_vec->data;
  for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
  {
    struct timeval start_section0, end_section0;
    gettimeofday(&start_section0, NULL);
    /* Begin section0 */
    for (int x = x_m; x <= x_M; x += 1)
    {
      for (int y = y_m; y <= y_M; y += 1)
      {
        u[t1][x + 1][y + 1] = u[t0][x + 1][y + 1] + 1;
      }
    }
    /* 

In [37]:
op.apply(time=2)
u.data

Operator `Kernel` run in 0.00 s


Data([[[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]],

      [[3., 3., 3.],
       [3., 3., 3.],
       [3., 3., 3.]]], dtype=float32)

In [38]:
from devito.tools import pprint
pprint(op)

<Callable Kernel>
  <List (0, 2, 0)>

    <ArrayCast>
    <List (0, 1, 0)>

      <[affine,sequential,wrappable] Iteration time::time::(time_m, time_M, 1)::(0, 0)>
        <TimedList (2, 1, 2)>
          <C.Statement struct timeval start_section0, end_section0;>
          <C.Statement gettimeofday(&start_section0, NULL);>
          <Section (1)>

            <[affine,parallel] Iteration x::x::(x_m, x_M, 1)::(0, 0)>
              <[affine,parallel,vector-dim] Iteration y::y::(y_m, y_M, 1)::(0, 0)>
                <ExpressionBundle (1)>

                  <Expression u[t1, x + 1, y + 1] = u[t0, x + 1, y + 1] + 1>


          <C.Statement gettimeofday(&end_section0, NULL);>
          <C.Statement timers->section0 += (double)(end_section0.tv_sec-start_section0.tv_sec)+(double)(end_section0.tv_usec-start_section0.tv_usec)/1000000;>




In [39]:
op._headers

['#define _POSIX_C_SOURCE 200809L']

In [40]:
op._includes

['stdlib.h', 'math.h', 'sys/time.h']

In [41]:
op.body

(<List (0, 2, 0)>,)

In [42]:
pprint(op.body)

<List (0, 2, 0)>

  <ArrayCast>
  <List (0, 1, 0)>

    <[affine,sequential,wrappable] Iteration time::time::(time_m, time_M, 1)::(0, 0)>
      <TimedList (2, 1, 2)>
        <C.Statement struct timeval start_section0, end_section0;>
        <C.Statement gettimeofday(&start_section0, NULL);>
        <Section (1)>

          <[affine,parallel] Iteration x::x::(x_m, x_M, 1)::(0, 0)>
            <[affine,parallel,vector-dim] Iteration y::y::(y_m, y_M, 1)::(0, 0)>
              <ExpressionBundle (1)>

                <Expression u[t1, x + 1, y + 1] = u[t0, x + 1, y + 1] + 1>


        <C.Statement gettimeofday(&end_section0, NULL);>
        <C.Statement timers->section0 += (double)(end_section0.tv_sec-start_section0.tv_sec)+(double)(end_section0.tv_usec-start_section0.tv_usec)/1000000;>




In [43]:
op_callable = op.body[0].body

In [44]:
print(op_callable[0])  # Printing the ArrayCast

float (*restrict u)[u_vec->size[1]][u_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]]) u_vec->data;


In [45]:
print(op_callable[1][0])  # Printing the List

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  for (int x = x_m; x <= x_M; x += 1)
  {
    for (int y = y_m; y <= y_M; y += 1)
    {
      u[t1][x + 1][y + 1] = u[t0][x + 1][y + 1] + 1;
    }
  }
  /* End section0 */
  gettimeofday(&end_section0, NULL);
  timers->section0 += (double)(end_section0.tv_sec-start_section0.tv_sec)+(double)(end_section0.tv_usec-start_section0.tv_usec)/1000000;
}


In [50]:
t_iter = op_callable[1][0].body[0] # Iteration representing the time loop
t_iter

<WithProperties[affine,sequential,wrappable]::Iteration time[t0,t1]; (time_m, time_M, 1)>

In [53]:
t_iter.limits

(time_m, time_M, 1)

In [54]:
# Exercise: find the IET's Expression!

expr = t_iter.nodes[0].body[0].body[0].nodes[0].nodes[0].body[0]
expr.view

'<Expression u[t1, x + 1, y + 1] = u[t0, x + 1, y + 1] + 1>'

In [55]:
from devito.ir.iet import Expression, FindNodes
exprs = FindNodes(Expression).visit(op)
exprs[0].view

'<Expression u[t1, x + 1, y + 1] = u[t0, x + 1, y + 1] + 1>'

# Bottom Up

In [106]:
from devito import SpaceDimension, TimeDimension

i = SpaceDimension(name='i')
j = SpaceDimension(name='j')
k = SpaceDimension(name='k')

In [109]:
from devito.types import Array, Scalar
from devito import Constant

a = Scalar(name='a')
b = Constant(name='b')
c = Array(name='c', shape=(3,), dimensions=(i,)).indexify()

In [110]:
from devito.ir.iet import Expression
from devito.ir.equations import DummyEq
from devito.tools import pprint

exprs = [Expression(DummyEq(a, a + b)),
         Expression(DummyEq(b, a + c + 5.))]


pprint(exprs)

<Expression a = a + b>
<Expression b = a + c[i] + 5.0>


In [115]:
from devito.ir.iet import Iteration

iters = [lambda ex: Iteration(ex, i, (0,3,1)),
         lambda ex: Iteration(ex, j, (0,5,1)),
         lambda ex: Iteration(ex, k, (0,7,1))]

iter1

<function __main__.<lambda>(ex)>

In [120]:
# Perfect loop nest:
# for i
#   for j
#     for k
#       expr0  
block1 = iters[0](iters[1](iters[2](exprs[0])))

pprint(block1)

<Iteration i::i::(0, 3, 1)::(0, 0)>
  <Iteration j::j::(0, 5, 1)::(0, 0)>
    <Iteration k::k::(0, 7, 1)::(0, 0)>
      <Expression a = a + b>


In [122]:
# Exercise:   

# Non-perfect simple loop nest:
# for i
#   expr0
#   for j
#     for k
#       expr1

#block2 = iters[0]([exprs[0], iters[1](iters[2](exprs[1]))])
#pprint(block2)

<Iteration i::i::(0, 3, 1)::(0, 0)>
  <Expression a = a + b>
  <Iteration j::j::(0, 5, 1)::(0, 0)>
    <Iteration k::k::(0, 7, 1)::(0, 0)>
      <Expression b = a + c[i] + 5.0>


In [123]:
from devito.ir.iet import Callable

kernel = Callable('my_kernel', block1, 'void', ())

print(str(kernel.ccode))

void my_kernel()
{
  for (int i = 0; i <= 3; i += 1)
  {
    for (int j = 0; j <= 5; j += 1)
    {
      for (int k = 0; k <= 7; k += 1)
      {
        a = a + b;
      }
    }
  }
}


In [90]:
from devito.ir.iet import Transformer

# Replaces an expression with another
transformer = Transformer({exprs[0]: exprs[1]})
newblock = transformer.visit(block1)
newcode = str(newblock.ccode)
print(newcode)

for (int i = 0; i <= 3; i += 1)
{
  for (int j = 0; j <= 5; j += 1)
  {
    for (int k = 0; k <= 7; k += 1)
    {
      b = a + c[i] + 5.0F;
    }
  }
}
